In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
%matplotlib inline

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
pdf = PdfPages("/home/c/cslage/u/AuxTel/dome/Shutter_State_05Feb25.pdf")
startDay = 20250101
endDay = 20250129
fig = plt.figure(figsize=(15,5))
dayObs = startDay
while dayObs <= endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    if dayObs == 20240529 or (dayObs > 20240530 and dayObs < 20240624):
        dayObs = calcNextDay(dayObs)
        continue
        
    try:
        start = Time(f"{dayString}T18:00:00", scale='utc')
        end = Time(f"{nextDayString}T11:00:00", scale='utc')
    
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)
        shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['mainDoorOpeningPercentage'],  start, end)
        close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                                    ['*'],  start, end)
        open_command = await client.select_time_series('lsst.sal.ATDome.command_openShutter', \
                                                    ['*'],  start, end)
        summary_state = await client.select_time_series('lsst.sal.ATDome.logevent_summaryState', \
                                                    ['*'],  start, end)

        dome_move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
        link = await client.select_time_series('lsst.sal.ATDome.logevent_scbLink', \
                                                    ['*'],  start, end) 
        link = link[link['active'] == False] # Lost comm between Top Box and Main Box
        print(len(state), len(shut), len(close_command), len(summary_state), len(dome_move), len(link))
        if len(shut) == 0:
            dayObs = calcNextDay(dayObs)
            continue
        ax = fig.subplots(1,1)
        ax.set_title(f"Shutter_State_{dayObs}", fontsize=24)
        #state['state'].plot(ax=ax, color='blue')
        states = state['state'].values
        tstates = tstates = state.index
        ax.step(tstates, states, color='blue', label='Actual State', where='post')        
        
        ax.set_yticks([1,2,3,4,5,6,7])
    
        ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
        ax.set_ylabel("Door State")
        ax1 = ax.twinx()
        shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', label='Shutter Open Percentage')
        ax1.set_ylabel("Open percentage")
        ax1.set_ylim(-2,115)

        

        
        disabled = summary_state[summary_state['summaryState'] == 1]
        enabled = summary_state[summary_state['summaryState'] == 2]
        standby = summary_state[summary_state['summaryState'] == 5]
        fault = summary_state[summary_state['summaryState'] == 3]
    
        """
        disabled = Time("2024-02-12T12:00:00", scale='utc')
        enabled = Time("2024-02-12T18:44:00", scale='utc')
        ax1.axvline(disabled.isot, ls='--', color='cyan', label='DISABLED')
        ax1.axvline(enabled.isot, ls='--', color='orange', label='ENABLED')
        """
        for close in close_command.index[0:1]:
            ax1.axvline(close, ls='-.', color='green', label='Close Command')
        for open in open_command.index[0:1]:
            ax1.axvline(open, ls='-.', color='magenta', label='Open Command')
        for disable in disabled.index[0:1]:
            ax1.axvline(disable, ls='--', color='cyan', label='Disabled')
        for enable in enabled.index[0:1]:
            ax1.axvline(enable, lw=2, color='darkgreen', label='Enabled')
        for std in standby.index[0:1]:
            ax1.axvline(std, ls='--', color='orange', label='Standby')
        for flt in fault.index[0:1]:
            ax1.axvline(flt, ls='--', color='red', label='Fault')
        for lnk in link.index[0:1]:
            ax1.axvline(lnk, ls=':', color='black', label='Lost Comm')
        
        for move in dome_move.index[0:1]:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='Dome_Move')

        """        
        enables = []
        cscs = ['ATDomeTrajectory', 'ATMCS', 'ATPtg', 'ATAOS', 'ATHexapod', 'ATPneumatics']
        
        for csc in cscs:
            query = f'lsst.sal.{csc}.logevent_summaryState'
            these_states = await client.select_time_series(query, ['*'],  start, end)
            enable = these_states[these_states['summaryState'] == 2]
            print(csc, len(enable))
            enables.append(enable)
           
        for i, this_enable in enumerate(enables):
            for en in this_enable.index[0:1]:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label=cscs[i])
        
        for i, this_enable in enumerate(enables):
            for en in this_enable.index:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label='')
        """                        
    
        for close in close_command.index:
            ax1.axvline(close, ls='-.', color='green', label='')
        for open in open_command.index:
            ax1.axvline(open, ls='-.', color='magenta', label='')
        for disable in disabled.index:
            ax1.axvline(disable, ls='--', color='cyan', label='')
        for enable in enabled.index:
            ax1.axvline(enable,  color='darkgreen', lw=2, label='')
        for std in standby.index:
            ax1.axvline(std, ls='--', color='orange', label='')
        for flt in fault.index:
            ax1.axvline(flt, ls='--', color='red', label='')
        for move in dome_move.index:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='')
        for lnk in link.index:
            ax1.axvline(lnk, ls=':', color='black', label='')
            
    
    
        ax.legend(loc='lower left')
        ax1.legend(loc='upper left')
    
        #startPlot = "2024-07-17T01:30:00"
        #endPlot = "2024-07-17T05:45:00"
        #ax.set_xlim(startPlot, endPlot)
        #ax1.set_xlim(startPlot, endPlot)
        
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        plt.clf()
        continue
pdf.close()


In [ ]:
state['state']

# The one below includes the timebase errors.

In [ ]:
%matplotlib inline
pdf = PdfPages("/home/c/cslage/u/AuxTel/dome/Shutter_State_Multiple_08Feb24.pdf")
startDay = 20240208
endDay = 20240208
fig = plt.figure(figsize=(15,5))
dayObs = startDay
while dayObs <= endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    
    #try:
    bad_starts = []
    bad_ends = []
    start = Time(f"{dayString}T20:00:00", scale='utc')
    end = Time(f"{nextDayString}T10:00:00", scale='utc')

    state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                        ['*'],  start, end)
    shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                                ['mainDoorOpeningPercentage'],  start, end)
    close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                                ['*'],  start, end)
    open_command = await client.select_time_series('lsst.sal.ATDome.command_openShutter', \
                                                ['*'],  start, end)

    mount_position = await client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                                     ['cRIO_timestamp'], start, end)
                                                            
    cRIO_ts = mount_position["cRIO_timestamp"].values
    isBad = False
    for n, value in enumerate(cRIO_ts):
        if n == 0:
            continue
        diff  = value - cRIO_ts[n-1]
        if diff > 1.05:
            if not isBad:
                bad_starts.append(pd.Timestamp(Time(value, format='unix_tai').utc.isot))
                isBad = True
        else:
            if isBad:
                bad_ends.append(pd.Timestamp(Time(value, format='unix_tai').utc.isot))
                isBad = False

    
    print(len(state), len(shut), len(close_command))
    if len(shut) == 0:
        dayObs = calcNextDay(dayObs)
        continue
    ax = fig.subplots(1,1)
    ax.set_title(f"Shutter_State_{dayObs}", fontsize=24)
    state['state'].plot(ax=ax, color='blue', label='Actual State')
    ax.set_yticks([1,2,3,4,5,6,7])

    ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
    ax.set_ylabel("Door State")
    ax1 = ax.twinx()
    shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', ls='-.', label='Shutter Open Percentage')
    ax1.set_ylabel("Open percentage")
    ax1.set_ylim(-2,115)

    for close in close_command.index[0:1]:
        ax1.axvline(close, ls='-.', color='green', label='Close Command')
    for open in open_command.index[0:1]:
        ax1.axvline(open, ls='-.', color='magenta', label='Open Command')
    ax1.plot([bad_starts[0], bad_ends[0]], [20,20], color='green', lw=5, label='Timebase Error')


    for close in close_command.index:
        ax1.axvline(close, ls='-.', color='green', label='')
    for open in open_command.index:
        ax1.axvline(open, ls='-.', color='magenta', label='')

    for n in range(1, len(bad_starts)):
        ax1.plot([bad_starts[n], bad_ends[n]], [20,20], color='green', lw=5, label='')
    ax.legend(loc='lower left')
    ax1.legend(loc='upper left')

    
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
    dayObs = calcNextDay(dayObs)
    #except:
    #    dayObs = calcNextDay(dayObs)
    #    continue
pdf.close()


In [ ]:
ax.step?

In [ ]:
state.columns